# Schnorr Identification Protocol

## Introduction

The **Schnorr Identification Protocol** is a fundamental zero-knowledge proof (ZKP) allowing a prover to demonstrate knowledge of a discrete logarithm without revealing any information about the logarithm itself.

Formally, the prover convinces the verifier that they know a secret exponent $x$ satisfying:

$$
y = g^x \mod p
$$

without leaking any information about $x$.

---

## Mathematical Setting

The protocol is defined over a finite cyclic group $\mathbb{G}$ of prime order $q$, generated by an element $g$.  
Typically, $\mathbb{G}$ is a subgroup of $\mathbb{Z}_p^*$, where $p$ is a large prime such that $q \mid p-1$.

- $p$: a large prime
- $q$: a prime divisor of $p-1$
- $g$: generator of a subgroup of order $q$ in $\mathbb{Z}_p^*$
- $x \in \mathbb{Z}_q$: prover's secret
- $y = g^x \mod p$: prover's public key

The hardness of computing $x$ from $(g, y)$ is based on the **Discrete Logarithm Problem (DLP)**.

---

## Protocol Description

The Schnorr protocol consists of the following steps:

### 1. Commitment
Prover selects a random nonce $r \in \mathbb{Z}_q$, and computes:

$$
t = g^r \mod p
$$

Prover sends $t$ to the verifier.

---

### 2. Challenge
Verifier selects a random challenge $c \in \mathbb{Z}_q$ and sends $c$ to the prover.

---

### 3. Response
Prover computes the response:

$$
s = (r + c \cdot x) \mod q
$$

and sends $s$ to the verifier.

---

### 4. Verification
Verifier checks that:

$$
g^s \equiv t \cdot y^c \mod p
$$

If the equation holds, the verifier accepts the proof.

---

## Security Properties

- **Completeness**: An honest prover with knowledge of $x$ will always convince the verifier.
- **Soundness**: A cheating prover cannot convince the verifier without knowledge of $x$, except with negligible probability.
- **Zero-Knowledge**: The transcript $(t, c, s)$ can be simulated without knowledge of $x$, ensuring that the verifier learns nothing about $x$.

# Python Implementation

We implement the protocol manually, avoiding external cryptographic libraries.

---

## Setup

In [6]:
import random

def generate_parameters():
    # Use small safe primes for demonstration
    p = 23  # Prime
    q = 11  # Prime divisor of p-1
    g = 2   # Generator
    return p, q, g

def keygen(p, q, g):
    x = random.randint(1, q-1)  # Prover's secret
    y = pow(g, x, p)            # Public key
    return x, y

---

## Protocol Execution

In [7]:
def schnorr_prove(p, q, g, x):
    r = random.randint(0, q-1)
    t = pow(g, r, p)
    return r, t

def schnorr_response(p, q, g, x, r, c):
    s = (r + c * x) % q
    return s

def schnorr_verify(p, q, g, y, t, c, s):
    lhs = pow(g, s, p)
    rhs = (t * pow(y, c, p)) % p
    return lhs == rhs

## Worked Example

Let's work through a full protocol instance using small primes for easy verification.

In [8]:
p, q, g = generate_parameters()
x, y = keygen(p, q, g)

print(f"Public parameters: p = {p}, q = {q}, g = {g}")
print(f"Prover's secret: x = {x}")
print(f"Prover's public key: y = {y}")

# Prover computes commitment
r, t = schnorr_prove(p, q, g, x)
print(f"Prover sends commitment t = {t}")

# Verifier sends challenge
c = random.randint(0, q-1)
print(f"Verifier sends challenge c = {c}")

# Prover computes response
s = schnorr_response(p, q, g, x, r, c)
print(f"Prover sends response s = {s}")

# Verifier checks
result = schnorr_verify(p, q, g, y, t, c, s)
print(f"Verification result: {result}")

Public parameters: p = 23, q = 11, g = 2
Prover's secret: x = 3
Prover's public key: y = 8
Prover sends commitment t = 2
Verifier sends challenge c = 8
Prover sends response s = 3
Verification result: True


## Large Prime Simulation

For realistic security, use larger primes:

In [9]:
def generate_large_parameters():
    p = 208351617316091241234326746312124448251235562226470491514186331217050270460481
    q = 104175808658045620617163373156062224125617781113235245757093165608525135230240
    g = 2
    return p, q, g

p, q, g = generate_large_parameters()
x, y = keygen(p, q, g)

r, t = schnorr_prove(p, q, g, x)
c = random.randint(0, q-1)
s = schnorr_response(p, q, g, x, r, c)

assert schnorr_verify(p, q, g, y, t, c, s)
print("Large prime verification succeeded.")

Large prime verification succeeded.
